In [3]:
import pandas as pd
from sqlalchemy import create_engine, text

# Connection setup
engine = create_engine(
    "oracle+oracledb://",
    connect_args={
        "user": "raw_layer",
        "password": "Raw#123",
        "dsn": "localhost:1521/XEPDB1",
    },
    pool_pre_ping=True,
)

with engine.begin() as conn:
    # ✅ Query all tables visible to the current user
    query = text("""
        SELECT table_name
        FROM user_tables
        ORDER BY table_name
    """)
    tables = pd.read_sql(query, conn)
    print("📋 Tables found:")
    print(tables)

📋 Tables found:
       table_name
0        BRANCHES
1            CARS
2  CAR_CATEGORIES
3       CUSTOMERS
4      IOT_ALERTS
5     IOT_DEVICES
6        MANAGERS
7        PAYMENTS
8         RENTALS
9    RESERVATIONS


In [4]:
from sqlalchemy import text

truncate_order = [
    "MANAGERS",     # dépend de BRANCHES
    "CARS",         # dépend de BRANCHES
    "RESERVATIONS", # dépend de CARS/BRANCHES
    "RENTALS",      # dépend de BRANCHES
    "PAYMENTS",     # dépend de RENTALS/RESERVATIONS
    "IOT_ALERTS",   # dépend de BRANCHES
    "BRANCHES"      # parent final
]

with engine.begin() as conn:
    for table in truncate_order:
        try:
            conn.execute(text(f"TRUNCATE TABLE {table}"))
            print(f"✅ Truncated {table}")
        except Exception as e:
            print(f"⚠️ Could not truncate {table}: {e}")

✅ Truncated MANAGERS
✅ Truncated CARS
✅ Truncated RESERVATIONS
✅ Truncated RENTALS
✅ Truncated PAYMENTS
✅ Truncated IOT_ALERTS
✅ Truncated BRANCHES


In [232]:
# === 1) Define branch data ===
branches_data = [
    ("Casablanca HQ",   "Bd Al Massira, Maarif",           "Casablanca", "+212522000111", "casa.hq@carrental.ma"),
    ("Rabat Agdal",     "Av. de France, Agdal",            "Rabat",      "+212537000222", "rabat.agdal@carrental.ma"),
    ("Marrakech Gueliz","Av. Mohammed V, Gueliz",          "Marrakech",  "+212524000333", "marrakech.gueliz@carrental.ma"),
    ("Tanger Downtown", "Rue de la Liberté, Centre-ville", "Tanger",     "+212539000444", "tanger.dt@carrental.ma"),
    ("Agadir Plage",    "Corniche, Plage",                 "Agadir",     "+212602555666", "agadir.plage@carrental.ma"),
]

columns = ["BRANCH_NAME", "ADDRESS", "CITY", "PHONE", "EMAIL"]
branches_df = pd.DataFrame(branches_data, columns=columns)

# === 2) Insert into Oracle ===
with engine.begin() as conn:
    branches_df.to_sql("BRANCHES", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(branches_df)} branches into BRANCHES table.")

✅ Inserted 5 branches into BRANCHES table.


C:\Users\hamza\AppData\Local\Temp\ipykernel_9404\2121359499.py:15: UserWarning: The provided table name 'BRANCHES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  branches_df.to_sql("BRANCHES", conn, if_exists="append", index=False)


In [5]:
BRANCHES  = pd.read_sql("SELECT * FROM BRANCHES", engine)
BRANCHES.head()

,branch_id,branch_name,address,city,phone,email,created_at


In [234]:
# --- 0) Truncate MANAGERS table (safe cleanup) ---
with engine.begin() as conn:
    try:
        conn.execute(text("TRUNCATE TABLE MANAGERS"))
        print("🧹 MANAGERS table truncated.")
    except Exception as e:
        print(f"⚠️ Could not truncate MANAGERS: {e}")

# --- 1) Load branches and build a name -> id map (case-robust) ---
with engine.begin() as conn:
    branches = pd.read_sql(
        text("SELECT BRANCH_ID AS branch_id, BRANCH_NAME AS branch_name FROM BRANCHES"),
        conn
    )

branches.columns = branches.columns.str.upper()

if branches.empty:
    raise RuntimeError("No rows in BRANCHES table — insert branches first.")

required = {"BRANCH_ID", "BRANCH_NAME"}
missing_cols = required - set(branches.columns)
if missing_cols:
    raise RuntimeError(f"Expected columns missing from BRANCHES: {missing_cols}. Got {list(branches.columns)}")

name_to_id = dict(zip(branches["BRANCH_NAME"].astype(str),
                      branches["BRANCH_ID"].astype(int)))

# --- 2) Define two managers per branch ---
raw_managers = [
    ("MGR101","Amina","Berrada","amina.berrada@carrental.ma","+212600100101","pwd#Casa1","Casablanca HQ"),
    ("MGR102","Karim","Saidi","karim.saidi@carrental.ma","+212600100102","pwd#Casa2","Casablanca HQ"),
    ("MGR201","Yassin","El Idrissi","yassin.elidrissi@carrental.ma","+212600200201","pwd#Rabat1","Rabat Agdal"),
    ("MGR202","Lina","Mouline","lina.mouline@carrental.ma","+212600200202","pwd#Rabat2","Rabat Agdal"),
    ("MGR301","Nadia","Zerouali","nadia.zerouali@carrental.ma","+212600300301","pwd#Mrk1","Marrakech Gueliz"),
    ("MGR302","Omar","Kabbaj","omar.kabbaj@carrental.ma","+212600300302","pwd#Mrk2","Marrakech Gueliz"),
    ("MGR401","Soukaina","Benali","soukaina.benali@carrental.ma","+212600400401","pwd#Tgr1","Tanger Downtown"),
    ("MGR402","Hicham","Alaoui","hicham.alaoui@carrental.ma","+212600400402","pwd#Tgr2","Tanger Downtown"),
    ("MGR501","Sara","El Fassi","sara.elfassi@carrental.ma","+212600500501","pwd#Agd1","Agadir Plage"),
    ("MGR502","Youssef","Boukhriss","youssef.boukhriss@carrental.ma","+212600500502","pwd#Agd2","Agadir Plage"),
]

cols = ["MANAGER_CODE","FIRST_NAME","LAST_NAME","EMAIL","PHONE","MANAGER_PASSWORD","BRANCH_NAME"]
df = pd.DataFrame(raw_managers, columns=cols)

# --- 3) Map branch names to IDs ---
unknown = sorted(set(df["BRANCH_NAME"]) - set(name_to_id.keys()))
if unknown:
    raise RuntimeError(f"Unknown branch names in manager list: {unknown}")

df["BRANCH_ID"] = df["BRANCH_NAME"].map(name_to_id)
managers_df = df[[
    "MANAGER_CODE","FIRST_NAME","LAST_NAME","EMAIL","PHONE","MANAGER_PASSWORD","BRANCH_ID"
]]

# --- 4) Insert into MANAGERS table ---
with engine.begin() as conn:
    managers_df.to_sql("MANAGERS", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(managers_df)} managers into MANAGERS.")


🧹 MANAGERS table truncated.
✅ Inserted 10 managers into MANAGERS.


C:\Users\hamza\AppData\Local\Temp\ipykernel_9404\1569102751.py:58: UserWarning: The provided table name 'MANAGERS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  managers_df.to_sql("MANAGERS", conn, if_exists="append", index=False)


In [235]:
MANAGERS  = pd.read_sql("SELECT * FROM MANAGERS", engine)
MANAGERS.head()

,manager_id,manager_code,first_name,last_name,email,phone,manager_password,hire_date,branch_id
0,21,MGR101,Amina,Berrada,amina.berrada@carrental.ma,+212600100101,pwd#Casa1,2025-10-20 19:36:02,11
1,22,MGR102,Karim,Saidi,karim.saidi@carrental.ma,+212600100102,pwd#Casa2,2025-10-20 19:36:02,11
2,23,MGR201,Yassin,El Idrissi,yassin.elidrissi@carrental.ma,+212600200201,pwd#Rabat1,2025-10-20 19:36:02,12
3,24,MGR202,Lina,Mouline,lina.mouline@carrental.ma,+212600200202,pwd#Rabat2,2025-10-20 19:36:02,12
4,25,MGR301,Nadia,Zerouali,nadia.zerouali@carrental.ma,+212600300301,pwd#Mrk1,2025-10-20 19:36:02,13


In [236]:
# --- 0) Truncate CAR_CATEGORIES (cleanup) ---
with engine.begin() as conn:
    try:
        conn.execute(text("TRUNCATE TABLE CAR_CATEGORIES"))
        print("🧹 CAR_CATEGORIES table truncated.")
    except Exception as e:
        print(f"⚠️ Could not truncate CAR_CATEGORIES: {e}")

# --- 1) Define 5 main car categories ---
categories = [
    ("Economy", "Small city cars; fuel-efficient and affordable"),
    ("SUV", "Sport Utility Vehicles; spacious and powerful"),
    ("Luxury", "Premium sedans and coupes; high comfort"),
    ("Van", "7–9 seat vehicles for families or groups"),
    ("Electric", "Fully electric vehicles; zero emissions"),
]

categories_df = pd.DataFrame(categories, columns=["CATEGORY_NAME", "DESCRIPTION"])

# --- 2) Insert into CAR_CATEGORIES ---
with engine.begin() as conn:
    categories_df.to_sql("CAR_CATEGORIES", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(categories_df)} categories into CAR_CATEGORIES.")


🧹 CAR_CATEGORIES table truncated.
✅ Inserted 5 categories into CAR_CATEGORIES.


C:\Users\hamza\AppData\Local\Temp\ipykernel_9404\2233146216.py:22: UserWarning: The provided table name 'CAR_CATEGORIES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  categories_df.to_sql("CAR_CATEGORIES", conn, if_exists="append", index=False)


In [237]:
CAR_CATEGORIES  = pd.read_sql("SELECT * FROM CAR_CATEGORIES", engine)
CAR_CATEGORIES.head()

,category_id,category_name,description,created_at
0,11,Economy,Small city cars; fuel-efficient and affordable,2025-10-20 19:36:07.098480
1,12,SUV,Sport Utility Vehicles; spacious and powerful,2025-10-20 19:36:07.098480
2,13,Luxury,Premium sedans and coupes; high comfort,2025-10-20 19:36:07.098480
3,14,Van,7–9 seat vehicles for families or groups,2025-10-20 19:36:07.098480
4,15,Electric,Fully electric vehicles; zero emissions,2025-10-20 19:36:07.098480


In [238]:
# --- 0) Truncate IOT_DEVICES before insert ---
with engine.begin() as conn:
    try:
        conn.execute(text("TRUNCATE TABLE IOT_DEVICES"))
        print("🧹 IOT_DEVICES table truncated.")
    except Exception as e:
        print(f"⚠️ Could not truncate IOT_DEVICES: {e}")

# --- 1) Define 10 inactive IoT devices ---
devices = [
    ("DEV001", "IMEI1000000001", "v1.2.0"),
    ("DEV002", "IMEI1000000002", "v1.2.0"),
    ("DEV003", "IMEI1000000003", "v1.1.5"),
    ("DEV004", "IMEI1000000004", "v1.3.0"),
    ("DEV005", "IMEI1000000005", "v1.3.1"),
    ("DEV006", "IMEI1000000006", "v1.2.1"),
    ("DEV007", "IMEI1000000007", "v1.0.9"),
    ("DEV008", "IMEI1000000008", "v1.4.0"),
    ("DEV009", "IMEI1000000009", "v1.4.0"),
    ("DEV010", "IMEI1000000010", "v1.5.0"),
]

columns = ["DEVICE_CODE", "DEVICE_IMEI", "FIRMWARE_VERSION"]
iot_df = pd.DataFrame(devices, columns=columns)

# Add default values for required columns
iot_df["STATUS"] = "INACTIVE"
iot_df["ACTIVATED_AT"] = None
iot_df["LAST_SEEN_AT"] = None

# --- 2) Insert into Oracle ---
with engine.begin() as conn:
    iot_df.to_sql("IOT_DEVICES", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(iot_df)} inactive IoT devices into IOT_DEVICES.")

🧹 IOT_DEVICES table truncated.
✅ Inserted 10 inactive IoT devices into IOT_DEVICES.


C:\Users\hamza\AppData\Local\Temp\ipykernel_9404\556460841.py:33: UserWarning: The provided table name 'IOT_DEVICES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  iot_df.to_sql("IOT_DEVICES", conn, if_exists="append", index=False)


In [239]:
IOT_DEVICES  = pd.read_sql("SELECT * FROM IOT_DEVICES", engine)
IOT_DEVICES.head()

,device_id,device_code,device_imei,firmware_version,status,activated_at,last_seen_at,created_at
0,21,DEV001,IMEI1000000001,v1.2.0,INACTIVE,None,None,2025-10-20 19:36:10.143204
1,22,DEV002,IMEI1000000002,v1.2.0,INACTIVE,None,None,2025-10-20 19:36:10.143204
2,23,DEV003,IMEI1000000003,v1.1.5,INACTIVE,None,None,2025-10-20 19:36:10.143204
3,24,DEV004,IMEI1000000004,v1.3.0,INACTIVE,None,None,2025-10-20 19:36:10.143204
4,25,DEV005,IMEI1000000005,v1.3.1,INACTIVE,None,None,2025-10-20 19:36:10.143204


In [240]:
import pandas as pd
from sqlalchemy import create_engine, text

# --- Connection ---
engine = create_engine(
    "oracle+oracledb://",
    connect_args={
        "user": "raw_layer",
        "password": "Raw#123",
        "dsn": "localhost:1521/XEPDB1",
    },
    pool_pre_ping=True,
)

# --- 0) Truncate CARS ---
with engine.begin() as conn:
    conn.execute(text("TRUNCATE TABLE CARS"))
    print("🧹 Table CARS truncated successfully.")

# --- 1) Maps (case-robust) ---
with engine.begin() as conn:
    cats = pd.read_sql(text("""
        SELECT CATEGORY_ID AS category_id, CATEGORY_NAME AS category_name
        FROM CAR_CATEGORIES
    """), conn)
    branches = pd.read_sql(text("""
        SELECT BRANCH_ID AS branch_id, BRANCH_NAME AS branch_name
        FROM BRANCHES
    """), conn)

cats.columns = cats.columns.str.upper()
branches.columns = branches.columns.str.upper()

if cats.empty:
    raise RuntimeError("CAR_CATEGORIES is empty. Seed categories first.")
if branches.empty:
    raise RuntimeError("BRANCHES is empty. Seed branches first.")

cat_map = dict(zip(cats["CATEGORY_NAME"].astype(str), cats["CATEGORY_ID"].astype(int)))
branch_map = dict(zip(branches["BRANCH_NAME"].astype(str), branches["BRANCH_ID"].astype(int)))

# --- 2) Cars to insert ---
cars_raw = [
    ("Economy",  "VIN000000001","A-101-CN","Dacia","Sandero", 2022,"White", 12000,"AVAILABLE","Casablanca HQ"),
    ("Economy",  "VIN000000002","A-102-CN","Toyota","Yaris",  2021,"Blue",  23000,"AVAILABLE","Casablanca HQ"),
    ("SUV",      "VIN000000003","B-201-SV","Hyundai","Tucson",2023,"Gray",   8000,"AVAILABLE","Rabat Agdal"),
    ("SUV",      "VIN000000004","B-202-SV","Kia","Sportage",  2022,"Black", 15000,"AVAILABLE","Rabat Agdal"),
    ("Luxury",   "VIN000000005","C-301-LX","BMW","530i",      2020,"Black", 42000,"AVAILABLE","Marrakech Gueliz"),
    ("Luxury",   "VIN000000006","C-302-LX","Mercedes","E200", 2021,"Silver",35000,"AVAILABLE","Marrakech Gueliz"),
    ("Van",      "VIN000000007","D-401-VN","Renault","Trafic",2019,"White", 68000,"AVAILABLE","Tanger Downtown"),
    ("Van",      "VIN000000008","D-402-VN","Ford","Tourneo",  2020,"Blue",  54000,"AVAILABLE","Tanger Downtown"),
    ("Electric", "VIN000000009","E-501-EV","Renault","Zoe",   2022,"Green",  9000,"AVAILABLE","Agadir Plage"),
    ("Electric", "VIN000000010","E-502-EV","Peugeot","e-208", 2023,"Yellow", 4000,"AVAILABLE","Agadir Plage"),
]
cols = ["CATEGORY_NAME","VIN","LICENSE_PLATE","MAKE","MODEL","MODEL_YEAR","COLOR","ODOMETER_KM","STATUS","BRANCH_NAME"]
df = pd.DataFrame(cars_raw, columns=cols)

# --- 3) Resolve FKs ---
df["CATEGORY_ID"] = df["CATEGORY_NAME"].map(cat_map)
df["BRANCH_ID"]   = df["BRANCH_NAME"].map(branch_map)

# --- 4) Get free INACTIVE devices (case-robust column) ---
with engine.begin() as conn:
    devices_free = pd.read_sql(text("""
        SELECT d.DEVICE_ID AS device_id
        FROM IOT_DEVICES d
        LEFT JOIN CARS c ON c.DEVICE_ID = d.DEVICE_ID
        WHERE d.STATUS = 'INACTIVE'
          AND c.DEVICE_ID IS NULL
        ORDER BY d.DEVICE_ID
    """), conn)

devices_free.columns = devices_free.columns.str.upper()
free_list = devices_free["DEVICE_ID"].astype(int).tolist() if not devices_free.empty else []

# --- 5) Assign 1 distinct device per car, else None ---
assigned = [free_list.pop(0) if free_list else None for _ in range(len(df))]
df["DEVICE_ID"] = assigned

# --- 6) Build final frame and insert ---
cars_df = df[[
    "CATEGORY_ID","DEVICE_ID","VIN","LICENSE_PLATE","MAKE","MODEL",
    "MODEL_YEAR","COLOR","ODOMETER_KM","STATUS","BRANCH_ID"
]]

with engine.begin() as conn:
    cars_df.to_sql("CARS", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(cars_df)} cars into CARS with device assignment (inactive or NULL).")

# --- 7) Summary ---
print("📋 Device assignment summary (None = no device available):")
print(df[["VIN","LICENSE_PLATE","DEVICE_ID","BRANCH_NAME","CATEGORY_NAME"]].reset_index(drop=True))


🧹 Table CARS truncated successfully.
✅ Inserted 10 cars into CARS with device assignment (inactive or NULL).
📋 Device assignment summary (None = no device available):
            VIN LICENSE_PLATE  DEVICE_ID       BRANCH_NAME CATEGORY_NAME
0  VIN000000001      A-101-CN         21     Casablanca HQ       Economy
1  VIN000000002      A-102-CN         22     Casablanca HQ       Economy
2  VIN000000003      B-201-SV         23       Rabat Agdal           SUV
3  VIN000000004      B-202-SV         24       Rabat Agdal           SUV
4  VIN000000005      C-301-LX         25  Marrakech Gueliz        Luxury
5  VIN000000006      C-302-LX         26  Marrakech Gueliz        Luxury
6  VIN000000007      D-401-VN         27   Tanger Downtown           Van
7  VIN000000008      D-402-VN         28   Tanger Downtown           Van
8  VIN000000009      E-501-EV         29      Agadir Plage      Electric
9  VIN000000010      E-502-EV         30      Agadir Plage      Electric


C:\Users\hamza\AppData\Local\Temp\ipykernel_9404\3693320309.py:87: UserWarning: The provided table name 'CARS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  cars_df.to_sql("CARS", conn, if_exists="append", index=False)


In [246]:
CARS  = pd.read_sql("SELECT * FROM CARS", engine)
CARS.head(10)

,car_id,category_id,device_id,vin,license_plate,make,model,model_year,color,odometer_km,status,branch_id,created_at
0,21,11,21,VIN000000001,A-101-CN,Dacia,Sandero,2022,White,12000,AVAILABLE,11,2025-10-20 19:36:12.465944
1,22,11,22,VIN000000002,A-102-CN,Toyota,Yaris,2021,Blue,23000,AVAILABLE,11,2025-10-20 19:36:12.465944
2,23,12,23,VIN000000003,B-201-SV,Hyundai,Tucson,2023,Gray,8000,AVAILABLE,12,2025-10-20 19:36:12.465944
3,24,12,24,VIN000000004,B-202-SV,Kia,Sportage,2022,Black,15000,AVAILABLE,12,2025-10-20 19:36:12.465944
4,25,13,25,VIN000000005,C-301-LX,BMW,530i,2020,Black,42000,AVAILABLE,13,2025-10-20 19:36:12.465944
5,26,13,26,VIN000000006,C-302-LX,Mercedes,E200,2021,Silver,35000,AVAILABLE,13,2025-10-20 19:36:12.465944
6,27,14,27,VIN000000007,D-401-VN,Renault,Trafic,2019,White,68000,AVAILABLE,14,2025-10-20 19:36:12.465944
7,28,14,28,VIN000000008,D-402-VN,Ford,Tourneo,2020,Blue,54000,AVAILABLE,14,2025-10-20 19:36:12.465944
8,29,15,29,VIN000000009,E-501-EV,Renault,Zoe,2022,Green,9000,AVAILABLE,15,2025-10-20 19:36:12.465944
9,30,15,30,VIN000000010,E-502-EV,Peugeot,e-208,2023,Yellow,4000,AVAILABLE,15,2025-10-20 19:36:12.465944


In [244]:
import pandas as pd
from sqlalchemy import create_engine, text

# --- Connexion ---
engine = create_engine(
    "oracle+oracledb://",
    connect_args={
        "user": "raw_layer",
        "password": "Raw#123",
        "dsn": "localhost:1521/XEPDB1",
    },
    pool_pre_ping=True,
)

# === Paramètres du client à créer ===
first_name = "yadsser"
last_name  = "elbdouk"
email      = "yasdser.elbdouk@example.com"   # peut être None si tu veux
phone      = "+21260486483456"
id_number  = "CIN-AS-0004"

# 1) Contrôles d’existence (idempotence légère)
with engine.begin() as conn:
    # par email (UNIQUE en base)
    if email:
        dup = pd.read_sql(
            text("SELECT CUSTOMER_ID FROM CUSTOMERS WHERE EMAIL = :em"),
            conn, params={"em": email}
        )
        if not dup.empty:
            customer_id = int(dup.iloc[0]["CUSTOMER_ID"])
            print(f"ℹ️ Client déjà présent (email) → CUSTOMER_ID = {customer_id}")
        else:
            customer_id = None
    else:
        customer_id = None

    # par ID_NUMBER (pas unique en DDL, mais on prévient les doublons courants)
    if customer_id is None and id_number:
        dup2 = pd.read_sql(
            text("SELECT CUSTOMER_ID FROM CUSTOMERS WHERE ID_NUMBER = :idn"),
            conn, params={"idn": id_number}
        )
        if not dup2.empty:
            customer_id = int(dup2.iloc[0]["CUSTOMER_ID"])
            print(f"ℹ️ Client déjà présent (ID_NUMBER) → CUSTOMER_ID = {customer_id}")

# 2) Insertion (si nécessaire) + RETURNING CUSTOMER_ID
if customer_id is None:
    with engine.begin() as conn:
        # On utilise le curseur Oracle pour RETURNING INTO
        raw = conn.connection
        cur = raw.cursor()
        rid = cur.var(int)

        cur.execute("""
            INSERT INTO CUSTOMERS (FIRST_NAME, LAST_NAME, EMAIL, PHONE, ID_NUMBER)
            VALUES (:fn, :ln, :em, :ph, :idn)
            RETURNING CUSTOMER_ID INTO :rid
        """, fn=first_name, ln=last_name, em=email, ph=phone, idn=id_number, rid=rid)

        customer_id = rid.getvalue()[0]
        print(f"✅ Client créé → CUSTOMER_ID = {customer_id}")

# 3) Affichage de la fiche client
with engine.begin() as conn:
    row = pd.read_sql(
        text("""
            SELECT CUSTOMER_ID, FIRST_NAME, LAST_NAME, EMAIL, PHONE, ID_NUMBER, CREATED_AT
            FROM CUSTOMERS
            WHERE CUSTOMER_ID = :cid
        """),
        conn, params={"cid": customer_id}
    )

print("📋 Fiche client :")
print(row)

✅ Client créé → CUSTOMER_ID = 4
📋 Fiche client :
   customer_id first_name last_name                        email  \
0            4    yadsser   elbdouk  yasdser.elbdouk@example.com   

             phone    id_number                 created_at  
0  +21260486483456  CIN-AS-0004 2025-10-20 19:37:19.938382  


In [245]:
CUSTOMERS  = pd.read_sql("SELECT * FROM CUSTOMERS", engine)
CUSTOMERS.head()

,customer_id,first_name,last_name,email,phone,id_number,created_at
0,1,Adil,Sabir,adil.sabir@example.com,+212600123456,CIN-AS-0001,2025-10-20 18:21:44.724922
1,2,hamza,bjibji,hamza.bjibji@example.com,+212604823456,CIN-AS-0002,2025-10-20 18:23:55.470606
2,3,yasser,elbouk,yasser.elbouk@example.com,+2126048483456,CIN-AS-0003,2025-10-20 18:29:39.874013
3,4,yadsser,elbdouk,yasdser.elbdouk@example.com,+21260486483456,CIN-AS-0004,2025-10-20 19:37:19.938382


In [248]:
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import datetime, timedelta

engine = create_engine(
    "oracle+oracledb://",
    connect_args={"user":"raw_layer","password":"Raw#123","dsn":"localhost:1521/XEPDB1"},
    pool_pre_ping=True,
)

# --- 0) Récupérer des voitures AVAILABLE (jusqu'à 6) ---
with engine.begin() as conn:
    cars = pd.read_sql(text("""
        SELECT 
          CAR_ID        AS car_id,
          BRANCH_ID     AS branch_id,
          LICENSE_PLATE AS license_plate,
          MAKE          AS make,
          MODEL         AS model
        FROM CARS
        WHERE STATUS = 'AVAILABLE'
        ORDER BY CAR_ID
        FETCH FIRST 6 ROWS ONLY
    """), conn)
cars.columns = cars.columns.str.upper()

if cars.empty:
    raise RuntimeError("Aucune voiture AVAILABLE. Seed/MAJ CARS d'abord.")

# --- 1) Un client (n'importe lequel) ---
with engine.begin() as conn:
    cust = pd.read_sql(text("""
        SELECT CUSTOMER_ID AS customer_id
        FROM CUSTOMERS
        ORDER BY CUSTOMER_ID
        FETCH FIRST 1 ROWS ONLY
    """), conn)
cust.columns = cust.columns.str.upper()
if cust.empty:
    raise RuntimeError("Aucun client trouvé. Créez d'abord au moins un client.")

customer_id = int(cust.loc[0, "CUSTOMER_ID"])

# --- 2) Préparer 6 plages de réservation (non chevauchantes) ---
base = datetime.now() + timedelta(hours=2)  # commence dans 2h
slots = [
    (base + timedelta(hours=0),  base + timedelta(hours=4),  300.00),
    (base + timedelta(hours=5),  base + timedelta(hours=9),  350.00),
    (base + timedelta(hours=10), base + timedelta(hours=14), 280.00),
    (base + timedelta(hours=15), base + timedelta(hours=19), 320.00),
    (base + timedelta(hours=20), base + timedelta(hours=26), 600.00),
    (base + timedelta(hours=27), base + timedelta(hours=33), 540.00),
]

created_ids = []

# --- 3) Boucle d'insertion ---
for i, (_, row) in enumerate(cars.iterrows()):
    car_id        = int(row["CAR_ID"])
    res_branch_id = int(row["BRANCH_ID"])  # branche de la voiture

    start_at, end_at, amount = slots[i % len(slots)]

    # Trouver un manager d'une autre branche
    with engine.begin() as conn:
        mgr = pd.read_sql(text("""
            SELECT MANAGER_ID AS manager_id
            FROM MANAGERS
            WHERE BRANCH_ID <> :bid
            ORDER BY MANAGER_ID
            FETCH FIRST 1 ROWS ONLY
        """), conn, params={"bid": res_branch_id})
    mgr.columns = mgr.columns.str.upper()
    manager_id = None if mgr.empty else int(mgr.loc[0, "MANAGER_ID"])

    # Vérifier disponibilité par la fonction
    with engine.begin() as conn:
        avail = pd.read_sql(text("""
            SELECT FN_CAR_AVAILABLE(:car_id, :s, :e) AS AV
            FROM DUAL
        """), conn, params={"car_id": car_id, "s": start_at, "e": end_at})
    avail.columns = avail.columns.str.upper()
    if int(avail.loc[0, "AV"]) != 1:
        print(f"⏭️  Car {car_id} non disponible {start_at} → {end_at}, on saute.")
        continue

    # Insert + RETURNING
    with engine.begin() as conn:
        raw = conn.connection
        cur = raw.cursor()
        rid = cur.var(int)

        cur.execute("""
            INSERT INTO RESERVATIONS (
                CAR_ID, CUSTOMER_ID, BRANCH_ID, MANAGER_ID,
                START_AT, END_AT, STATUS, TOTAL_AMOUNT, CURRENCY, NOTES
            ) VALUES (
                :car_id, :cust_id, :branch_id, :mgr_id,
                :start_at, :end_at, 'PENDING', :amount, 'MAD', :notes
            )
            RETURNING RESERVATION_ID INTO :rid
        """,
        car_id=car_id,
        cust_id=customer_id,
        branch_id=res_branch_id,              # branche de la voiture
        mgr_id=manager_id,                    # manager autre branche (ou NULL)
        start_at=start_at,
        end_at=end_at,
        amount=amount,
        notes=f"Bulk insert test {i+1}: manager from different branch",
        rid=rid)

        reservation_id = rid.getvalue()[0]
        created_ids.append(reservation_id)
        print(f"✅ Reservation #{reservation_id} insérée | CAR={car_id} | BRANCH={res_branch_id} | MGR={manager_id} | {start_at}→{end_at}")

# --- 4) Récap ---
if created_ids:
    with engine.begin() as conn:
        recap = pd.read_sql(text(f"""
            SELECT r.RESERVATION_ID,
                   r.STATUS,
                   r.START_AT, r.END_AT, r.TOTAL_AMOUNT,
                   r.BRANCH_ID AS RES_BRANCH_ID,
                   r.MANAGER_ID,
                   (SELECT BRANCH_ID FROM MANAGERS m WHERE m.MANAGER_ID = r.MANAGER_ID) AS MGR_BRANCH_ID,
                   r.CAR_ID
            FROM RESERVATIONS r
            WHERE r.RESERVATION_ID IN ({",".join([str(x) for x in created_ids])})
            ORDER BY r.RESERVATION_ID
        """), conn)
    recap.columns = recap.columns.str.upper()
    print("\n📋 Récap des réservations créées :")
    print(recap)

    # Vérifier la contrainte "manager d'une autre branche" quand MANAGER_ID n'est pas NULL
    check = recap.dropna(subset=["MANAGER_ID"])
    if not check.empty:
        ok = (check["RES_BRANCH_ID"].astype(int) != check["MGR_BRANCH_ID"].astype(int)).all()
        print("✅ Manager branch ≠ reservation branch pour tous les cas avec manager." if ok else "⚠️ Incohérence manager/branche détectée.")
else:
    print("ℹ️ Aucune réservation insérée (voitures indisponibles ou autre condition).")

✅ Reservation #7 insérée | CAR=21 | BRANCH=11 | MGR=23 | 2025-10-20 22:38:09.529631→2025-10-21 02:38:09.529631
✅ Reservation #8 insérée | CAR=22 | BRANCH=11 | MGR=23 | 2025-10-21 03:38:09.529631→2025-10-21 07:38:09.529631
✅ Reservation #9 insérée | CAR=23 | BRANCH=12 | MGR=21 | 2025-10-21 08:38:09.529631→2025-10-21 12:38:09.529631
✅ Reservation #10 insérée | CAR=24 | BRANCH=12 | MGR=21 | 2025-10-21 13:38:09.529631→2025-10-21 17:38:09.529631
✅ Reservation #11 insérée | CAR=25 | BRANCH=13 | MGR=21 | 2025-10-21 18:38:09.529631→2025-10-22 00:38:09.529631
✅ Reservation #12 insérée | CAR=26 | BRANCH=13 | MGR=21 | 2025-10-22 01:38:09.529631→2025-10-22 07:38:09.529631

📋 Récap des réservations créées :
   RESERVATION_ID   STATUS            START_AT              END_AT  \
0               7  PENDING 2025-10-20 22:38:09 2025-10-21 02:38:09   
1               8  PENDING 2025-10-21 03:38:09 2025-10-21 07:38:09   
2               9  PENDING 2025-10-21 08:38:09 2025-10-21 12:38:09   
3              1

In [249]:
RESERVATIONS  = pd.read_sql("SELECT * FROM RESERVATIONS", engine)
RESERVATIONS.head(10)

,reservation_id,car_id,customer_id,branch_id,manager_id,start_at,end_at,status,total_amount,currency,notes,created_at
0,7,21,1,11,23,2025-10-20 22:38:09,2025-10-21 02:38:09,PENDING,300.0,MAD,Bulk insert test 1: manager from different branch,2025-10-20 19:38:09.577036
1,8,22,1,11,23,2025-10-21 03:38:09,2025-10-21 07:38:09,PENDING,350.0,MAD,Bulk insert test 2: manager from different branch,2025-10-20 19:38:09.602951
2,9,23,1,12,21,2025-10-21 08:38:09,2025-10-21 12:38:09,PENDING,280.0,MAD,Bulk insert test 3: manager from different branch,2025-10-20 19:38:09.612176
3,10,24,1,12,21,2025-10-21 13:38:09,2025-10-21 17:38:09,PENDING,320.0,MAD,Bulk insert test 4: manager from different branch,2025-10-20 19:38:09.622158
4,11,25,1,13,21,2025-10-21 18:38:09,2025-10-22 00:38:09,PENDING,600.0,MAD,Bulk insert test 5: manager from different branch,2025-10-20 19:38:09.630834
5,12,26,1,13,21,2025-10-22 01:38:09,2025-10-22 07:38:09,PENDING,540.0,MAD,Bulk insert test 6: manager from different branch,2025-10-20 19:38:09.640045


In [250]:
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import datetime, timedelta

engine = create_engine(
    "oracle+oracledb://",
    connect_args={"user":"raw_layer","password":"Raw#123","dsn":"localhost:1521/XEPDB1"},
    pool_pre_ping=True,
)

# --- 1) Récupérer jusqu'à 5 voitures AVAILABLE ---
with engine.begin() as conn:
    cars = pd.read_sql(text("""
        SELECT 
          CAR_ID        AS car_id,
          BRANCH_ID     AS branch_id,
          LICENSE_PLATE AS license_plate,
          ODOMETER_KM   AS odometer_km
        FROM CARS
        WHERE STATUS = 'AVAILABLE'
        ORDER BY CAR_ID
        FETCH FIRST 5 ROWS ONLY
    """), conn)
cars.columns = cars.columns.str.upper()

if cars.empty:
    raise RuntimeError("Aucune voiture AVAILABLE. Seed/MAJ CARS d'abord.")

# --- 2) Choisir un client (n'importe lequel) ---
with engine.begin() as conn:
    cust = pd.read_sql(text("""
        SELECT CUSTOMER_ID AS customer_id
        FROM CUSTOMERS
        ORDER BY CUSTOMER_ID
        FETCH FIRST 1 ROWS ONLY
    """), conn)
cust.columns = cust.columns.str.upper()
if cust.empty:
    raise RuntimeError("Aucun client trouvé. Créez d'abord au moins un client.")

customer_id = int(cust.loc[0, "CUSTOMER_ID"])

# --- 3) Insertion de rentals "walk-in" (RESERVATION_ID = NULL) ---
created_rentals = []
now = datetime.now()

for i, row in cars.iterrows():
    car_id     = int(row["CAR_ID"])
    branch_id  = int(row["BRANCH_ID"])
    start_odo  = int(row["ODOMETER_KM"] or 0)

    # Fenêtres horaires décalées pour éviter les overlaps
    start_at = now + timedelta(hours=1 + i*2)   # commence dans 1h, puis +2h à chaque voiture
    due_at   = start_at + timedelta(hours=6)    # durée 6h

    # Manager de la même agence si possible (sinon NULL)
    with engine.begin() as conn:
        mgr = pd.read_sql(text("""
            SELECT MANAGER_ID AS manager_id
            FROM MANAGERS
            WHERE BRANCH_ID = :bid
            ORDER BY MANAGER_ID
            FETCH FIRST 1 ROWS ONLY
        """), conn, params={"bid": branch_id})
    mgr.columns = mgr.columns.str.upper()
    manager_id = None if mgr.empty else int(mgr.loc[0, "MANAGER_ID"])

    # (Optionnel) Vérifier la dispo avant d'insérer (utile même en walk-in)
    with engine.begin() as conn:
        avail = pd.read_sql(text("""
            SELECT FN_CAR_AVAILABLE(:car_id, :s, :e) AS AV
            FROM DUAL
        """), conn, params={"car_id": car_id, "s": start_at, "e": due_at})
    avail.columns = avail.columns.str.upper()
    if int(avail.loc[0, "AV"]) != 1:
        print(f"⏭️  Car {car_id} indisponible {start_at} → {due_at}, on saute.")
        continue

    # INSERT dans RENTALS (RESERVATION_ID = NULL) + RETURNING RENTAL_ID
    try:
        with engine.begin() as conn:
            raw = conn.connection
            cur = raw.cursor()
            out_id = cur.var(int)

            cur.execute("""
                INSERT INTO RENTALS(
                  RESERVATION_ID, CAR_ID, CUSTOMER_ID, BRANCH_ID, MANAGER_ID,
                  START_AT, DUE_AT, STATUS, START_ODOMETER, CURRENCY
                ) VALUES (
                  NULL, :car_id, :cust_id, :branch_id, :mgr_id,
                  :start_at, :due_at, 'ACTIVE', :start_odo, 'MAD'
                )
                RETURNING RENTAL_ID INTO :rid
            """,
            car_id=car_id,
            cust_id=customer_id,
            branch_id=branch_id,
            mgr_id=manager_id,
            start_at=start_at,
            due_at=due_at,
            start_odo=start_odo,
            rid=out_id)

            rental_id = int(out_id.getvalue()[0])
            created_rentals.append(rental_id)
            print(f"✅ Rental #{rental_id} créé | CAR={car_id} | BR={branch_id} | MGR={manager_id} | ODO={start_odo} | {start_at}→{due_at}")

    except Exception as e:
        print(f"⚠️  Échec création rental pour CAR {car_id}: {e}")

# --- 4) Récap + contrôle statut voiture ---
if created_rentals:
    ids_csv = ",".join(str(x) for x in created_rentals)
    with engine.begin() as conn:
        recap = pd.read_sql(text(f"""
            SELECT
              l.RENTAL_ID,
              l.CAR_ID,
              l.CUSTOMER_ID,
              l.BRANCH_ID,
              l.MANAGER_ID,
              l.START_AT,
              l.DUE_AT,
              l.RETURN_AT,
              l.STATUS           AS RENTAL_STATUS,
              l.START_ODOMETER,
              l.END_ODOMETER,
              c.LICENSE_PLATE,
              c.STATUS           AS CAR_STATUS
            FROM RENTALS l
            JOIN CARS c ON c.CAR_ID = l.CAR_ID
            WHERE l.RENTAL_ID IN ({ids_csv})
            ORDER BY l.RENTAL_ID
        """), conn)

    print("\n📋 Récap rentals créés :")
    print(recap)

    # Alerte si la voiture n'est pas passée en RENTED (les triggers doivent le faire)
    bad = recap[recap["CAR_STATUS"] != "RENTED"]
    if not bad.empty:
        print("\n⚠️ Certaines voitures ne sont pas en statut RENTED (à inspecter):")
        print(bad[["RENTAL_ID","CAR_ID","LICENSE_PLATE","CAR_STATUS"]])
else:
    print("ℹ️ Aucun rental créé.")


⏭️  Car 21 indisponible 2025-10-20 21:38:16.364164 → 2025-10-21 03:38:16.364164, on saute.
⏭️  Car 22 indisponible 2025-10-20 23:38:16.364164 → 2025-10-21 05:38:16.364164, on saute.
✅ Rental #4 créé | CAR=23 | BR=12 | MGR=23 | ODO=8000 | 2025-10-21 01:38:16.364164→2025-10-21 07:38:16.364164
✅ Rental #5 créé | CAR=24 | BR=12 | MGR=23 | ODO=15000 | 2025-10-21 03:38:16.364164→2025-10-21 09:38:16.364164
✅ Rental #6 créé | CAR=25 | BR=13 | MGR=25 | ODO=42000 | 2025-10-21 05:38:16.364164→2025-10-21 11:38:16.364164

📋 Récap rentals créés :
   rental_id  car_id  customer_id  branch_id  manager_id            start_at  \
0          4      23            1         12          23 2025-10-21 01:38:16   
1          5      24            1         12          23 2025-10-21 03:38:16   
2          6      25            1         13          25 2025-10-21 05:38:16   

               due_at return_at rental_status  start_odometer end_odometer  \
0 2025-10-21 07:38:16      None        ACTIVE            8000 

KeyError: 'CAR_STATUS'

In [251]:
RENTALS  = pd.read_sql("SELECT * FROM RENTALS", engine)
RENTALS.head(10)

,rental_id,reservation_id,car_id,customer_id,branch_id,manager_id,start_at,due_at,return_at,status,start_odometer,end_odometer,total_amount,currency,created_at
0,4,None,23,1,12,23,2025-10-21 01:38:16,2025-10-21 07:38:16,None,ACTIVE,8000,None,None,MAD,2025-10-20 19:38:16.388192
1,5,None,24,1,12,23,2025-10-21 03:38:16,2025-10-21 09:38:16,None,ACTIVE,15000,None,None,MAD,2025-10-20 19:38:16.400172
2,6,None,25,1,13,25,2025-10-21 05:38:16,2025-10-21 11:38:16,None,ACTIVE,42000,None,None,MAD,2025-10-20 19:38:16.412415


In [2]:
# 01_seed_static.py
# -------------------------------------------------------------------
# Full static seed with hard reset, robust sequence reset, and seeding:
# - Wipes all business tables (children -> parents)
# - Resets Oracle identity/classic sequences (XE-safe)
# - Seeds BRANCHES, MANAGERS, CAR_CATEGORIES, IOT_DEVICES, CARS
# -------------------------------------------------------------------

import pandas as pd
from sqlalchemy import create_engine, text

# ----------------------------
# Connection
# ----------------------------
engine = create_engine(
    "oracle+oracledb://",
    connect_args={"user": "raw_layer", "password": "Raw#123", "dsn": "localhost:1521/XEPDB1"},
    pool_pre_ping=True,
)

# ----------------------------
# Helpers: truncate / delete / identity reset
# ----------------------------
TRUNCATE_ORDER = [
    # Children first
    "PAYMENTS",
    "RENTALS",
    "RESERVATIONS",
    "IOT_ALERTS",
    "MANAGERS",
    "CARS",
    "IOT_DEVICES",
    "CAR_CATEGORIES",
    "CUSTOMERS",
    "BRANCHES",  # Parents last
]

def try_truncate_or_delete(conn, table):
    try:
        conn.execute(text(f"TRUNCATE TABLE {table}"))
        print(f"🧹 TRUNCATE {table}")
        return
    except Exception as e:
        print(f"⚠️  TRUNCATE {table} failed → {e}; trying DELETE ...")
    deleted = conn.execute(text(f"DELETE FROM {table}")).rowcount
    print(f"🧽 DELETE {table}: {deleted} rows")

def restart_identities(conn):
    """
    Reset identity and classic sequences. Robust to XE column-name quirks.
    Strategy:
      1) USER_TAB_IDENTITY_COLS → SEQUENCE_NAME (if present)
      2) USER_SEQUENCES where SEQUENCE_NAME like 'ISEQ$$_%'
      3) (Dev-only fallback) all USER_SEQUENCES
      4) Try ALTER SEQUENCE ... RESTART; else do "increment trick"
    """
    # 1) Identity sequences (preferred)
    seq_names = []
    try:
        rows = pd.read_sql(text("SELECT * FROM USER_TAB_IDENTITY_COLS"), conn)
        if not rows.empty:
            rows.columns = [c.upper().strip() for c in rows.columns]
            if "SEQUENCE_NAME" in rows.columns:
                seq_names = rows["SEQUENCE_NAME"].dropna().astype(str).tolist()
    except Exception as e:
        print(f"ℹ️ USER_TAB_IDENTITY_COLS not accessible: {e}")

    # 2) Identity-like sequences if none found
    if not seq_names:
        try:
            idseqs = pd.read_sql(
                text("SELECT SEQUENCE_NAME FROM USER_SEQUENCES WHERE SEQUENCE_NAME LIKE 'ISEQ$$_%' ORDER BY SEQUENCE_NAME"),
                conn
            )
            idseqs.columns = [c.upper().strip() for c in idseqs.columns]
            if "SEQUENCE_NAME" in idseqs.columns:
                seq_names = idseqs["SEQUENCE_NAME"].astype(str).tolist()
        except Exception as e:
            print(f"ℹ️ USER_SEQUENCES (ISEQ) not accessible: {e}")

    # 3) (Optional) all sequences as last resort
    if not seq_names:
        try:
            allseqs = pd.read_sql(text("SELECT SEQUENCE_NAME FROM USER_SEQUENCES ORDER BY SEQUENCE_NAME"), conn)
            allseqs.columns = [c.upper().strip() for c in allseqs.columns]
            if "SEQUENCE_NAME" in allseqs.columns:
                seq_names = allseqs["SEQUENCE_NAME"].astype(str).tolist()
        except Exception as e:
            print(f"ℹ️ USER_SEQUENCES not accessible: {e}")

    if not seq_names:
        print("ℹ️ No sequences found to reset; skipping.")
        return

    for seq in seq_names:
        try:
            conn.execute(text(f"ALTER SEQUENCE {seq} RESTART START WITH 1"))
            print(f"🔁 RESET sequence {seq} → 1")
        except Exception as e:
            # Version-safe fallback: "increment trick"
            try:
                last = pd.read_sql(
                    text("SELECT LAST_NUMBER FROM USER_SEQUENCES WHERE SEQUENCE_NAME = :s"),
                    conn, params={"s": seq}
                )
                last.columns = [c.upper().strip() for c in last.columns]
                if not last.empty and "LAST_NUMBER" in last.columns:
                    last_num = int(last.iloc[0]["LAST_NUMBER"])
                    delta = 1 - last_num
                    conn.execute(text(f"ALTER SEQUENCE {seq} INCREMENT BY {delta}"))
                    _ = pd.read_sql(text(f"SELECT {seq}.NEXTVAL AS NV FROM DUAL"), conn)
                    conn.execute(text(f"ALTER SEQUENCE {seq} INCREMENT BY 1"))
                    print(f"🔁 RESET sequence {seq} via increment trick → 1")
                else:
                    print(f"⚠️ Could not read LAST_NUMBER for {seq}: {e}")
            except Exception as e2:
                print(f"⚠️ Could not reset sequence {seq}: {e2}")

def wipe_all_data():
    with engine.begin() as conn:
        for t in TRUNCATE_ORDER:
            try_truncate_or_delete(conn, t)
        restart_identities(conn)
    print("🧨 Database data wiped (business tables).")

# ----------------------------
# Utility mappers / metadata
# ----------------------------
def map_table(conn, sql, key_col, val_col):
    t = pd.read_sql(text(sql), conn)
    if t.empty:
        return {}
    t.columns = [c.upper() for c in t.columns]
    return dict(zip(t[key_col.upper()].astype(str), t[val_col.upper()].astype(int)))

def _get_iot_device_pk_name(conn):
    # Prefer identity column if defined for IOT_DEVICES
    idx = pd.read_sql(
        text("SELECT COLUMN_NAME FROM USER_TAB_IDENTITY_COLS WHERE TABLE_NAME = 'IOT_DEVICES'"),
        conn
    )
    if not idx.empty and "COLUMN_NAME" in [c.upper() for c in idx.columns]:
        col = str(idx.iloc[0][idx.columns[0]]).strip()
        return col

    # Else inspect columns, prefer DEVICE_ID if present
    cols = pd.read_sql(
        text("SELECT COLUMN_NAME FROM USER_TAB_COLUMNS WHERE TABLE_NAME = 'IOT_DEVICES'"),
        conn
    )
    if cols.empty:
        raise RuntimeError("IOT_DEVICES not found or no columns visible.")
    names = [str(c).upper().strip() for c in cols["COLUMN_NAME"].tolist()]
    if "DEVICE_ID" in names:
        return "DEVICE_ID"
    return names[0]  # last resort

def _fetch_free_device_ids(conn, device_pk_col):
    # Devices INACTIVE and not linked to any CARS.DEVICE_ID
    q = f"""
        SELECT d.{device_pk_col} AS DEVICE_PK
        FROM IOT_DEVICES d
        LEFT JOIN CARS c ON c.DEVICE_ID = d.{device_pk_col}
        WHERE d.STATUS = 'INACTIVE' AND c.DEVICE_ID IS NULL
        ORDER BY d.{device_pk_col}
    """
    t = pd.read_sql(text(q), conn)
    t.columns = [c.upper().strip() for c in t.columns]
    return t["DEVICE_PK"].astype(int).tolist() if not t.empty else []

# ----------------------------
# Seeders
# ----------------------------
def seed_branches():
    data = [
        ("Casablanca HQ",   "Bd Al Massira, Maarif",           "Casablanca", "+212522000111", "casa.hq@carrental.ma"),
        ("Rabat Agdal",     "Av. de France, Agdal",            "Rabat",      "+212537000222", "rabat.agdal@carrental.ma"),
        ("Marrakech Gueliz","Av. Mohammed V, Gueliz",          "Marrakech",  "+212524000333", "marrakech.gueliz@carrental.ma"),
        ("Tanger Downtown", "Rue de la Liberté, Centre-ville", "Tanger",     "+212539000444", "tanger.dt@carrental.ma"),
        ("Agadir Plage",    "Corniche, Plage",                 "Agadir",     "+212602555666", "agadir.plage@carrental.ma"),
    ]
    df = pd.DataFrame(data, columns=["BRANCH_NAME","ADDRESS","CITY","PHONE","EMAIL"])
    with engine.begin() as conn:
        df.to_sql("BRANCHES", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df)} branches")

def seed_managers():
    rows = [
        ("MGR101","Amina","Berrada","amina.berrada@carrental.ma","+212600100101","pwd#Casa1","Casablanca HQ"),
        ("MGR102","Karim","Saidi","karim.saidi@carrental.ma","+212600100102","pwd#Casa2","Casablanca HQ"),
        ("MGR201","Yassin","El Idrissi","yassin.elidrissi@carrental.ma","+212600200201","pwd#Rabat1","Rabat Agdal"),
        ("MGR202","Lina","Mouline","lina.mouline@carrental.ma","+212600200202","pwd#Rabat2","Rabat Agdal"),
        ("MGR301","Nadia","Zerouali","nadia.zerouali@carrental.ma","+212600300301","pwd#Mrk1","Marrakech Gueliz"),
        ("MGR302","Omar","Kabbaj","omar.kabbaj@carrental.ma","+212600300302","pwd#Mrk2","Marrakech Gueliz"),
        ("MGR401","Soukaina","Benali","soukaina.benali@carrental.ma","+212600400401","pwd#Tgr1","Tanger Downtown"),
        ("MGR402","Hicham","Alaoui","hicham.alaoui@carrental.ma","+212600400402","pwd#Tgr2","Tanger Downtown"),
        ("MGR501","Sara","El Fassi","sara.elfassi@carrental.ma","+212600500501","pwd#Agd1","Agadir Plage"),
        ("MGR502","Youssef","Boukhriss","youssef.boukhriss@carrental.ma","+212600500502","pwd#Agd2","Agadir Plage"),
    ]
    df = pd.DataFrame(rows, columns=["MANAGER_CODE","FIRST_NAME","LAST_NAME","EMAIL","PHONE","MANAGER_PASSWORD","BRANCH_NAME"])
    with engine.begin() as conn:
        bmap = map_table(conn, "SELECT BRANCH_ID, BRANCH_NAME FROM BRANCHES", "BRANCH_NAME", "BRANCH_ID")
        if not bmap:
            raise RuntimeError("Seed branches first.")
        missing = sorted(set(df.BRANCH_NAME.unique()) - set(bmap.keys()))
        if missing:
            raise RuntimeError(f"Unknown branches for managers: {missing}")
        df["BRANCH_ID"] = df["BRANCH_NAME"].map(bmap)
        ins = df[["MANAGER_CODE","FIRST_NAME","LAST_NAME","EMAIL","PHONE","MANAGER_PASSWORD","BRANCH_ID"]]
        ins.to_sql("MANAGERS", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df)} managers")

def seed_categories():
    rows = [
        ("Economy",  "Small city cars; fuel-efficient and affordable"),
        ("SUV",      "Sport Utility Vehicles; spacious and powerful"),
        ("Luxury",   "Premium sedans and coupes; high comfort"),
        ("Van",      "7–9 seat vehicles for families or groups"),
        ("Electric", "Fully electric vehicles; zero emissions"),
    ]
    df = pd.DataFrame(rows, columns=["CATEGORY_NAME","DESCRIPTION"])
    with engine.begin() as conn:
        df.to_sql("CAR_CATEGORIES", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df)} car categories")

def seed_iot_devices():
    rows = [
        ("DEV001", "IMEI1000000001", "v1.2.0"),
        ("DEV002", "IMEI1000000002", "v1.2.0"),
        ("DEV003", "IMEI1000000003", "v1.1.5"),
        ("DEV004", "IMEI1000000004", "v1.3.0"),
        ("DEV005", "IMEI1000000005", "v1.3.1"),
        ("DEV006", "IMEI1000000006", "v1.2.1"),
        ("DEV007", "IMEI1000000007", "v1.0.9"),
        ("DEV008", "IMEI1000000008", "v1.4.0"),
        ("DEV009", "IMEI1000000009", "v1.4.0"),
        ("DEV010", "IMEI1000000010", "v1.5.0"),
    ]
    df = pd.DataFrame(rows, columns=["DEVICE_CODE","DEVICE_IMEI","FIRMWARE_VERSION"])
    df["STATUS"] = "INACTIVE"
    df["ACTIVATED_AT"] = None
    df["LAST_SEEN_AT"] = None
    with engine.begin() as conn:
        df.to_sql("IOT_DEVICES", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df)} IoT devices (INACTIVE)")

def seed_cars():
    rows = [
    # Casablanca HQ (15)
    ("Economy","VIN000000001","A-101-CN","Dacia","Sandero",2022,"White",12000,"AVAILABLE","Casablanca HQ"),
    ("Economy","VIN000000002","A-102-CN","Toyota","Yaris",2021,"Blue",23000,"AVAILABLE","Casablanca HQ"),
    ("Economy","VIN000000003","A-103-CN","Kia","Picanto",2023,"Red",8000,"AVAILABLE","Casablanca HQ"),
    ("SUV","VIN000000004","A-104-SV","Hyundai","Tucson",2023,"Gray",9000,"AVAILABLE","Casablanca HQ"),
    ("SUV","VIN000000005","A-105-SV","Nissan","Qashqai",2022,"Silver",11000,"AVAILABLE","Casablanca HQ"),
    ("Luxury","VIN000000006","A-106-LX","BMW","530i",2021,"Black",41000,"AVAILABLE","Casablanca HQ"),
    ("Luxury","VIN000000007","A-107-LX","Audi","A6",2022,"White",37000,"AVAILABLE","Casablanca HQ"),
    ("Van","VIN000000008","A-108-VN","Renault","Trafic",2020,"Gray",60000,"AVAILABLE","Casablanca HQ"),
    ("Van","VIN000000009","A-109-VN","Ford","Tourneo",2021,"Blue",52000,"AVAILABLE","Casablanca HQ"),
    ("Electric","VIN000000010","A-110-EV","Renault","Zoe",2022,"Green",9500,"AVAILABLE","Casablanca HQ"),
    ("Electric","VIN000000011","A-111-EV","Peugeot","e-208",2023,"Yellow",4000,"AVAILABLE","Casablanca HQ"),
    ("Electric","VIN000000012","A-112-EV","Tesla","Model 3",2023,"White",7000,"AVAILABLE","Casablanca HQ"),
    ("Economy","VIN000000013","A-113-CN","Fiat","Panda",2021,"Red",17000,"AVAILABLE","Casablanca HQ"),
    ("SUV","VIN000000014","A-114-SV","Dacia","Duster",2022,"Orange",21000,"AVAILABLE","Casablanca HQ"),
    ("Luxury","VIN000000015","A-115-LX","Mercedes","E200",2021,"Black",36000,"AVAILABLE","Casablanca HQ"),

    # Rabat Agdal (10)
    ("Economy","VIN000000016","B-201-CN","Toyota","Yaris",2022,"Gray",19000,"AVAILABLE","Rabat Agdal"),
    ("Economy","VIN000000017","B-202-CN","Hyundai","i10",2023,"Blue",8000,"AVAILABLE","Rabat Agdal"),
    ("SUV","VIN000000018","B-203-SV","Kia","Sportage",2022,"Black",15000,"AVAILABLE","Rabat Agdal"),
    ("SUV","VIN000000019","B-204-SV","Volkswagen","T-Roc",2021,"White",21000,"AVAILABLE","Rabat Agdal"),
    ("Luxury","VIN000000020","B-205-LX","BMW","320i",2022,"Blue",28000,"AVAILABLE","Rabat Agdal"),
    ("Luxury","VIN000000021","B-206-LX","Audi","A4",2023,"Silver",19000,"AVAILABLE","Rabat Agdal"),
    ("Van","VIN000000022","B-207-VN","Ford","Transit",2020,"White",65000,"AVAILABLE","Rabat Agdal"),
    ("Van","VIN000000023","B-208-VN","Mercedes","Vito",2021,"Gray",52000,"AVAILABLE","Rabat Agdal"),
    ("Electric","VIN000000024","B-209-EV","Nissan","Leaf",2022,"Green",10000,"AVAILABLE","Rabat Agdal"),
    ("Electric","VIN000000025","B-210-EV","Peugeot","e-2008",2023,"Black",6000,"AVAILABLE","Rabat Agdal"),

    # Marrakech Gueliz (10)
    ("Economy","VIN000000026","C-301-CN","Renault","Clio",2021,"Gray",22000,"AVAILABLE","Marrakech Gueliz"),
    ("SUV","VIN000000027","C-302-SV","Jeep","Compass",2022,"Red",17000,"AVAILABLE","Marrakech Gueliz"),
    ("SUV","VIN000000028","C-303-SV","Hyundai","Kona",2023,"Silver",9000,"AVAILABLE","Marrakech Gueliz"),
    ("Luxury","VIN000000029","C-304-LX","Mercedes","C-Class",2021,"Black",39000,"AVAILABLE","Marrakech Gueliz"),
    ("Luxury","VIN000000030","C-305-LX","BMW","X3",2022,"White",31000,"AVAILABLE","Marrakech Gueliz"),
    ("Van","VIN000000031","C-306-VN","Fiat","Ducato",2020,"White",72000,"AVAILABLE","Marrakech Gueliz"),
    ("Van","VIN000000032","C-307-VN","Peugeot","Expert",2021,"Gray",54000,"AVAILABLE","Marrakech Gueliz"),
    ("Electric","VIN000000033","C-308-EV","Tesla","Model Y",2023,"Blue",8000,"AVAILABLE","Marrakech Gueliz"),
    ("Electric","VIN000000034","C-309-EV","Renault","Megane E-Tech",2023,"Yellow",4000,"AVAILABLE","Marrakech Gueliz"),
    ("Economy","VIN000000035","C-310-CN","Suzuki","Swift",2022,"Orange",15000,"AVAILABLE","Marrakech Gueliz"),

    # Tanger Downtown (10)
    ("Economy","VIN000000036","D-401-CN","Dacia","Logan",2020,"White",45000,"AVAILABLE","Tanger Downtown"),
    ("SUV","VIN000000037","D-402-SV","Toyota","RAV4",2021,"Black",23000,"AVAILABLE","Tanger Downtown"),
    ("SUV","VIN000000038","D-403-SV","Kia","Seltos",2023,"Gray",9000,"AVAILABLE","Tanger Downtown"),
    ("Luxury","VIN000000039","D-404-LX","Audi","A5",2022,"Blue",21000,"AVAILABLE","Tanger Downtown"),
    ("Luxury","VIN000000040","D-405-LX","BMW","X5",2023,"Silver",18000,"AVAILABLE","Tanger Downtown"),
    ("Van","VIN000000041","D-406-VN","Mercedes","Vito",2020,"White",71000,"AVAILABLE","Tanger Downtown"),
    ("Van","VIN000000042","D-407-VN","Ford","Transit",2021,"Blue",65000,"AVAILABLE","Tanger Downtown"),
    ("Electric","VIN000000043","D-408-EV","Nissan","Leaf",2022,"Green",12000,"AVAILABLE","Tanger Downtown"),
    ("Electric","VIN000000044","D-409-EV","Peugeot","e-208",2023,"Red",5000,"AVAILABLE","Tanger Downtown"),
    ("Economy","VIN000000045","D-410-CN","Toyota","Aygo",2021,"Orange",18000,"AVAILABLE","Tanger Downtown"),

    # Agadir Plage (10)
    ("Economy","VIN000000046","E-501-CN","Hyundai","i20",2023,"White",9000,"AVAILABLE","Agadir Plage"),
    ("SUV","VIN000000047","E-502-SV","Nissan","Juke",2021,"Gray",25000,"AVAILABLE","Agadir Plage"),
    ("SUV","VIN000000048","E-503-SV","Kia","Seltos",2022,"Black",20000,"AVAILABLE","Agadir Plage"),
    ("Luxury","VIN000000049","E-504-LX","BMW","530e",2023,"Silver",12000,"AVAILABLE","Agadir Plage"),
    ("Luxury","VIN000000050","E-505-LX","Mercedes","C-Class",2022,"White",15000,"AVAILABLE","Agadir Plage"),
    ("Van","VIN000000051","E-506-VN","Peugeot","Traveller",2021,"Gray",61000,"AVAILABLE","Agadir Plage"),
    ("Van","VIN000000052","E-507-VN","Renault","Trafic",2020,"White",68000,"AVAILABLE","Agadir Plage"),
    ("Electric","VIN000000053","E-508-EV","Tesla","Model 3",2023,"Black",7000,"AVAILABLE","Agadir Plage"),
    ("Electric","VIN000000054","E-509-EV","Renault","Zoe",2022,"Blue",8000,"AVAILABLE","Agadir Plage"),
    ("Electric","VIN000000055","E-510-EV","Peugeot","e-208",2023,"Yellow",6000,"AVAILABLE","Agadir Plage"),
]

    df = pd.DataFrame(rows, columns=[
        "CATEGORY_NAME","VIN","LICENSE_PLATE","MAKE","MODEL","MODEL_YEAR","COLOR","ODOMETER_KM","STATUS","BRANCH_NAME"
    ])
    with engine.begin() as conn:
        # Maps
        cmap = map_table(conn, "SELECT CATEGORY_ID, CATEGORY_NAME FROM CAR_CATEGORIES", "CATEGORY_NAME", "CATEGORY_ID")
        bmap = map_table(conn, "SELECT BRANCH_ID, BRANCH_NAME FROM BRANCHES", "BRANCH_NAME", "BRANCH_ID")
        if not cmap or not bmap:
            raise RuntimeError("Seed categories and branches first.")

        # Detect PK column of IOT_DEVICES and pull free IDs
        device_pk = _get_iot_device_pk_name(conn)
        free = _fetch_free_device_ids(conn, device_pk)

        # Validate maps
        missing_cat = sorted(set(df.CATEGORY_NAME.unique()) - set(cmap.keys()))
        missing_br  = sorted(set(df.BRANCH_NAME.unique())   - set(bmap.keys()))
        if missing_cat: raise RuntimeError(f"Unknown categories for cars: {missing_cat}")
        if missing_br:  raise RuntimeError(f"Unknown branches for cars: {missing_br}")

        # Resolve FKs + device assignment
        df["CATEGORY_ID"] = df["CATEGORY_NAME"].map(cmap)
        df["BRANCH_ID"]   = df["BRANCH_NAME"].map(bmap)
        df["DEVICE_ID"]   = [free.pop(0) if free else None for _ in range(len(df))]

        ins = df[[
            "CATEGORY_ID","DEVICE_ID","VIN","LICENSE_PLATE","MAKE","MODEL",
            "MODEL_YEAR","COLOR","ODOMETER_KM","STATUS","BRANCH_ID"
        ]]
        ins.to_sql("CARS", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df)} cars with device assignment")

# ----------------------------
# Main
# ----------------------------
def main():
    wipe_all_data()     # full reset
    seed_branches()
    seed_managers()
    seed_categories()
    seed_iot_devices()
    seed_cars()
    print("🎉 Static seed completed (fresh database).")

if __name__ == "__main__":
    main()

🧹 TRUNCATE PAYMENTS
🧹 TRUNCATE RENTALS
🧹 TRUNCATE RESERVATIONS
🧹 TRUNCATE IOT_ALERTS
🧹 TRUNCATE MANAGERS
🧹 TRUNCATE CARS
🧹 TRUNCATE IOT_DEVICES
🧹 TRUNCATE CAR_CATEGORIES
🧹 TRUNCATE CUSTOMERS
🧹 TRUNCATE BRANCHES
⚠️ Could not reset sequence ISEQ$$_75936: (oracledb.exceptions.DatabaseError) ORA-32793: cannot alter a system-generated sequence
Help: https://docs.oracle.com/error-help/db/ora-32793/
[SQL: ALTER SEQUENCE ISEQ$$_75936 INCREMENT BY -40]
(Background on this error at: https://sqlalche.me/e/20/4xp6)
⚠️ Could not reset sequence ISEQ$$_75940: (oracledb.exceptions.DatabaseError) ORA-32793: cannot alter a system-generated sequence
Help: https://docs.oracle.com/error-help/db/ora-32793/
[SQL: ALTER SEQUENCE ISEQ$$_75940 INCREMENT BY -60]
(Background on this error at: https://sqlalche.me/e/20/4xp6)
⚠️ Could not reset sequence ISEQ$$_75944: (oracledb.exceptions.DatabaseError) ORA-32793: cannot alter a system-generated sequence
Help: https://docs.oracle.com/error-help/db/ora-32793/
[SQL: AL

C:\Users\hamza\AppData\Local\Temp\ipykernel_35436\1950620159.py:184: UserWarning: The provided table name 'BRANCHES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql("BRANCHES", conn, if_exists="append", index=False)
C:\Users\hamza\AppData\Local\Temp\ipykernel_35436\1950620159.py:210: UserWarning: The provided table name 'MANAGERS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  ins.to_sql("MANAGERS", conn, if_exists="append", index=False)
C:\Users\hamza\AppData\Local\Temp\ipykernel_35436\1950620159.py:223: UserWarning: The provided table name 'CAR_CATEGORIES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql("CAR_CATEGORIES", conn, if_exists="append", index=Fals

✅ Inserted 5 branches
✅ Inserted 10 managers
✅ Inserted 5 car categories
✅ Inserted 10 IoT devices (INACTIVE)
✅ Inserted 10 cars with device assignment
🎉 Static seed completed (fresh database).


C:\Users\hamza\AppData\Local\Temp\ipykernel_35436\1950620159.py:289: UserWarning: The provided table name 'CARS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  ins.to_sql("CARS", conn, if_exists="append", index=False)


In [1]:
# 02_simulate_daily_ops.py
import random
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine, text

# ----------------------------
# Config
# ----------------------------
RNG_SEED = 42
CLIENTS_PER_BRANCH_PER_DAY = 2
RESERVE_PROB = 0.5      # 50% reservations, 50% walk-in rentals
MIN_DURATION_H = 3
MAX_DURATION_H = 24
CURRENCY = "MAD"

# Date window (naive local times; Africa/Casablanca timezone in your env)
START_DATE = datetime.now().replace(hour=9, minute=0, second=0, microsecond=0)  # today 09:00
DAYS = 1  # change to simulate multiple days

random.seed(RNG_SEED)

engine = create_engine(
    "oracle+oracledb://",
    connect_args={
        "user": "raw_layer",
        "password": "Raw#123",
        "dsn": "localhost:1521/XEPDB1",
    },
    pool_pre_ping=True,
)

FIRST_NAMES = ["Youssef", "Aya", "Hamza", "Amal", "Mehdi", "Sara", "Karim", "Rania", "Anas", "Salma"]
LAST_NAMES  = ["El Fassi", "Bennani", "Alaoui", "El Idrissi", "Zerouali", "Kabbaj", "Berrada", "Saidi", "Benali", "Mouline"]

def pick_name_email_phone(i):
    fn = random.choice(FIRST_NAMES)
    ln = random.choice(LAST_NAMES)
    em = f"{fn.lower().replace(' ','')}.{ln.lower().replace(' ','')}.{int(datetime.now().timestamp())%100000+i}@example.com"
    ph = f"+2126{random.randint(10000000, 99999999)}"
    idn = f"CIN-{random.randint(100000, 999999)}"
    return fn, ln, em, ph, idn

def upsert_customer(fn, ln, em, ph, idn):
    with engine.begin() as conn:
        # check by email
        dup = pd.read_sql(text("SELECT CUSTOMER_ID FROM CUSTOMERS WHERE EMAIL = :em"), conn, params={"em": em})
        if not dup.empty:
            return int(dup.iloc[0]["CUSTOMER_ID"])

        # else insert with RETURNING
        raw = conn.connection
        cur = raw.cursor()
        rid = cur.var(int)
        cur.execute("""
            INSERT INTO CUSTOMERS (FIRST_NAME, LAST_NAME, EMAIL, PHONE, ID_NUMBER)
            VALUES (:fn, :ln, :em, :ph, :idn)
            RETURNING CUSTOMER_ID INTO :rid
        """, fn=fn, ln=ln, em=em, ph=ph, idn=idn, rid=rid)
        return int(rid.getvalue()[0])

def managers_in_branch(branch_id):
    with engine.begin() as conn:
        t = pd.read_sql(text("""
            SELECT MANAGER_ID FROM MANAGERS
            WHERE BRANCH_ID = :bid
            ORDER BY MANAGER_ID
        """), conn, params={"bid": branch_id})
    return t["MANAGER_ID"].astype(int).tolist() if not t.empty else []

def pick_available_car(branch_id, start_at, end_at):
    """Pick the first car in the branch that is available for the window."""
    with engine.begin() as conn:
        cars = pd.read_sql(text("""
            SELECT CAR_ID, ODOMETER_KM
            FROM CARS
            WHERE BRANCH_ID = :bid
            ORDER BY CAR_ID
        """), conn, params={"bid": branch_id})

    for _, r in cars.iterrows():
        car_id = int(r["CAR_ID"])
        with engine.begin() as conn:
            avail = pd.read_sql(text("""
                SELECT FN_CAR_AVAILABLE(:car_id, :s, :e) AS AV
                FROM DUAL
            """), conn, params={"car_id": car_id, "s": start_at, "e": end_at})
        if int(avail.iloc[0]["AV"]) == 1:
            return car_id, int(r["ODOMETER_KM"] or 0)
    return None, None

def create_reservation(car_id, customer_id, branch_id, manager_id, start_at, end_at, amount, notes):
    with engine.begin() as conn:
        raw = conn.connection
        cur = raw.cursor()
        rid = cur.var(int)
        cur.execute("""
            INSERT INTO RESERVATIONS (
              CAR_ID, CUSTOMER_ID, BRANCH_ID, MANAGER_ID,
              START_AT, END_AT, STATUS, TOTAL_AMOUNT, CURRENCY, NOTES
            ) VALUES (
              :car_id, :cust_id, :branch_id, :mgr_id,
              :start_at, :end_at, 'PENDING', :amount, :currency, :notes
            )
            RETURNING RESERVATION_ID INTO :rid
        """, car_id=car_id, cust_id=customer_id, branch_id=branch_id, mgr_id=manager_id,
           start_at=start_at, end_at=end_at, amount=amount, currency=CURRENCY, notes=notes, rid=rid)
        return int(rid.getvalue()[0])

def create_rental_walkin(car_id, customer_id, branch_id, manager_id, start_at, due_at, start_odo):
    with engine.begin() as conn:
        raw = conn.connection
        cur = raw.cursor()
        rid = cur.var(int)
        cur.execute("""
            INSERT INTO RENTALS(
              RESERVATION_ID, CAR_ID, CUSTOMER_ID, BRANCH_ID, MANAGER_ID,
              START_AT, DUE_AT, STATUS, START_ODOMETER, CURRENCY
            ) VALUES (
              NULL, :car_id, :cust_id, :branch_id, :mgr_id,
              :start_at, :due_at, 'ACTIVE', :start_odo, :currency
            )
            RETURNING RENTAL_ID INTO :rid
        """, car_id=car_id, cust_id=customer_id, branch_id=branch_id, mgr_id=manager_id,
           start_at=start_at, due_at=due_at, start_odo=start_odo, currency=CURRENCY, rid=rid)
        return int(rid.getvalue()[0])

def simulate_day(day_start):
    day_end = day_start.replace(hour=20, minute=0, second=0, microsecond=0)  # business window 09:00–20:00
    print(f"\n📅 Simulating {day_start.date()}")

    # branches
    with engine.begin() as conn:
        branches = pd.read_sql(text("SELECT BRANCH_ID, BRANCH_NAME FROM BRANCHES ORDER BY BRANCH_ID"), conn)

    if branches.empty:
        raise RuntimeError("No branches to simulate. Run 01_seed_static.py first.")

    created_res = 0
    created_ren = 0

    for _, b in branches.iterrows():
        bid = int(b["BRANCH_ID"])
        bname = b["BRANCH_NAME"]
        mgrs = managers_in_branch(bid)
        mgr_id = mgrs[0] if mgrs else None

        for i in range(CLIENTS_PER_BRANCH_PER_DAY):
            # time window per client
            start_at = day_start + timedelta(hours=random.randint(0, 8))  # between 09:00 and 17:00 start
            dur_h = random.randint(MIN_DURATION_H, MAX_DURATION_H)
            end_at = min(start_at + timedelta(hours=dur_h), day_end + timedelta(hours=6))  # allow late returns

            # customer
            fn, ln, em, ph, idn = pick_name_email_phone(i + bid * 10)
            cust_id = upsert_customer(fn, ln, em, ph, idn)

            # pick car available in this branch
            car_id, start_odo = pick_available_car(bid, start_at, end_at)
            if not car_id:
                print(f"⏭️  {bname}: no car available {start_at:%H:%M}→{end_at:%H:%M}")
                continue

            # decide reservation vs walk-in rental
            if random.random() < RESERVE_PROB:
                amount = float(random.randint(250, 900))
                rid = create_reservation(
                    car_id=car_id, customer_id=cust_id, branch_id=bid, manager_id=mgr_id,
                    start_at=start_at, end_at=end_at, amount=amount,
                    notes=f"Simulated reservation {bname}"
                )
                created_res += 1
                print(f"✅ Reservation #{rid} | BR={bid} | CAR={car_id} | {start_at:%H:%M}→{end_at:%H:%M} | {amount:.0f} {CURRENCY}")
            else:
                due_at = end_at
                lid = create_rental_walkin(
                    car_id=car_id, customer_id=cust_id, branch_id=bid, manager_id=mgr_id,
                    start_at=start_at, due_at=due_at, start_odo=start_odo
                )
                created_ren += 1
                print(f"🚗 Rental #{lid} | BR={bid} | CAR={car_id} | {start_at:%H:%M}→{due_at:%H:%M}")

    print(f"📊 Day summary: {created_res} reservations, {created_ren} rentals")

def main():
    for d in range(DAYS):
        simulate_day(START_DATE + timedelta(days=d))

    # Optional post-check: show latest inserts quick view
    with engine.begin() as conn:
        res = pd.read_sql(text("""
            SELECT RESERVATION_ID, BRANCH_ID, CAR_ID, START_AT, END_AT, STATUS, TOTAL_AMOUNT
            FROM RESERVATIONS
            WHERE START_AT >= :since
            ORDER BY RESERVATION_ID DESC FETCH FIRST 10 ROWS ONLY
        """), conn, params={"since": START_DATE - timedelta(days=DAYS)})
        ren = pd.read_sql(text("""
            SELECT RENTAL_ID, BRANCH_ID, CAR_ID, START_AT, DUE_AT, STATUS, START_ODOMETER
            FROM RENTALS
            WHERE START_AT >= :since
            ORDER BY RENTAL_ID DESC FETCH FIRST 10 ROWS ONLY
        """), conn, params={"since": START_DATE - timedelta(days=DAYS)})

    print("\n🧾 Last reservations:")
    print(res)
    print("\n🧾 Last rentals:")
    print(ren)

if __name__ == "__main__":
    main()



📅 Simulating 2025-10-21


KeyError: 'BRANCH_ID'